In [6]:
!pip3 install tensorflow

^C


     -------------------------------------- 276.5/276.5 MB 1.1 MB/s eta 0:00:00
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached h5py-3.9.0-cp39-cp39-win_amd64.whl (2.7 MB)
  Using cached libclang-16.0.6-py2.py3-none-win_amd64.whl (24.4 MB)
  Using cached numpy-1.24.3-cp39-cp39-win_amd64.whl (14.9 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached protobuf-4.24.1-cp39-cp39-win_amd64.whl (430 kB)
  Using cached termcolor-2.3.0-py3-none-any.whl (6.9 kB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Using cached wrapt-1.15.0-cp39-cp39-win_amd64.whl (36 kB)
  Using cached grpcio-1.57.0-cp39-cp39-win_amd64.whl (4.3 MB)
     ---------------------------------------- 5.6/5.6 MB 1.5 MB/s eta 0:00:0

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'd:\\programmfiles\\miniconda\\envs\\dev\\lib\\site-packages\\keras-2.13.1.dist-info\\METADATA'



In [23]:
import pandas as pd

# Загрузка данных
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Предобработка
train_data['Text'] = train_data['Text'].str.lower().str.replace('[^\w\s]', '')
test_data['Text'] = test_data['Text'].str.lower().str.replace('[^\w\s]', '')

train_data = train_data.drop('Unnamed: 0', axis=1).dropna()
print(train_data.describe())

                                        Text Sentiment
count                                  41155     41155
unique                                 41153         5
top     part 2 no toilet paper don t do this  Positive
freq                                       2     11422


In [24]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Преобразование текста в последовательности чисел
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(train_data['Text'])

train_sequences = tokenizer.texts_to_sequences(train_data['Text'])
test_sequences = tokenizer.texts_to_sequences(test_data['Text'])

# Дополнение последовательностей до одной и той же длины
train_padded = pad_sequences(train_sequences, maxlen=100, truncating='post', padding='post')
test_padded = pad_sequences(test_sequences, maxlen=100, truncating='post', padding='post')

In [28]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.utils import to_categorical

# Преобразование меток в числовые категории
train_labels = pd.get_dummies(train_data['Sentiment']).values
# test_labels = pd.get_dummies(test_data['Sentiment']).values

# Построение модели
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=64, input_length=100))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(5, activation='softmax'))  # 5 класса: Negative, Positive, Neutral, Extremely Negative, Extremely Positive

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Обучение модели
model.fit(train_padded, train_labels, epochs=5, batch_size=256, validation_split=0.2)

Epoch 1/5
129/129 [==============================] - 30s 215ms/step - loss: 1.5782 - accuracy: 0.2717 - val_loss: 1.5745 - val_accuracy: 0.2881
Epoch 2/5
129/129 [==============================] - 28s 216ms/step - loss: 1.5762 - accuracy: 0.2749 - val_loss: 1.5755 - val_accuracy: 0.2881
Epoch 3/5
129/129 [==============================] - 28s 217ms/step - loss: 1.5765 - accuracy: 0.2749 - val_loss: 1.5723 - val_accuracy: 0.2881
Epoch 4/5
129/129 [==============================] - 28s 219ms/step - loss: 1.5762 - accuracy: 0.2749 - val_loss: 1.5735 - val_accuracy: 0.2881
Epoch 5/5
129/129 [==============================] - 28s 220ms/step - loss: 1.5764 - accuracy: 0.2749 - val_loss: 1.5733 - val_accuracy: 0.2881


In [30]:
loss, accuracy = model.evaluate(test_padded)
print(f"Точность на тестовой выборке: {accuracy * 100:.2f}%")

119/119 [==============================] - 0s 822us/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Точность на тестовой выборке: 0.00%
